In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import json
import os
import sklearn as sk

In [5]:
classes_train = pd.read_csv('../data/train/_classes.csv')
classes_test = pd.read_csv('../data/test/_classes.csv')
classes_valid = pd.read_csv('../data/valid/_classes.csv')

# Concatenate all data
all_data_merged = pd.concat([classes_train, classes_test, classes_valid])

melted = all_data_merged.melt(id_vars=["filename"], var_name="category", value_name="value")
melted = melted[melted["value"] == 1]
all_data_merged_class = melted.drop("value", axis=1).rename(columns={"category": "class"})

all_data_merged_class.head()

subdatasets = [[] for _ in range(5)]

# Create a dictionary to map filenames to their full rows
filename_to_row = {row['filename']: row for _, row in all_data_merged.iterrows()}

# Distribute entries into the subdatasets
for idx, row in all_data_merged_class.iterrows():
    full_row = filename_to_row[row['filename']]
    subdatasets[idx % 5].append(full_row)

# Convert lists to DataFrames
subdatasets = [pd.DataFrame(subdataset) for subdataset in subdatasets]

# Verify the first few rows of each subdataset
for i, subdataset in enumerate(subdatasets):
    subdataset.sample(frac=1).reset_index(drop=True)

#shuffle the subdatasets
for i, subdataset in enumerate(subdatasets):
    subdatasets[i] = subdataset.sample(frac=1).reset_index(drop=True)
    



In [6]:
#write subdatasets to csv files in data/cross_valid_sets
for i, subdataset in enumerate(subdatasets):
    subdataset.to_csv(f'../data/cross_valid_sets/subdataset_{i}.csv', index=False)
    

In [7]:
test_1 = pd.read_csv('../data/cross_valid_sets/subdataset_1.csv')
test_2= pd.read_csv('../data/cross_valid_sets/subdataset_2.csv')
test_3 = pd.read_csv('../data/cross_valid_sets/subdataset_3.csv')

sum_1 = test_1[" SafeDriving"].sum()
sum_2 = test_1[" DangerousDriving"].sum()
sum_3 = test_1[" Yawn"].sum()

sum_4 = classes_train[" SafeDriving"].sum()
sum_5 = classes_train[" DangerousDriving"].sum()
sum_6 = classes_train[" Yawn"].sum()
print(sum_1, sum_2, sum_3)
print(sum_4, sum_5, sum_6)


1231 938 101
4969 3732 439


In [8]:
print(all_data_merged[" SafeDriving"].iloc[5])


0


In [9]:
from sklearn.model_selection import StratifiedKFold
X_aug = np.load('../data/data_aug_2000.npy')
y_aug = pd.read_csv('../data/labels_aug_2000.csv', delimiter=',', index_col=0).to_numpy()
print(y_aug[-1])
labels_aug = []

for i in range(len(y_aug)):
    # print(y_aug[i])
    if 1 in y_aug[i]:
        k = np.where(y_aug[i] == 1)[0][0]
        labels_aug.append(k)
    else:
        labels_aug.append(7)

labels_aug = np.array(labels_aug)
ind = np.where(labels_aug == 7)
print(ind)


kf = StratifiedKFold(n_splits=6, shuffle=True, random_state=42)




[0 0 0 1 0 0]
(array([ 276, 3947, 3961, 6886]),)


In [10]:
cross_ind = []

for train_index, val_index in kf.split(X = X_aug, y = labels_aug):
    # train_data, val_data = X_aug[train_index], X_aug[val_index]
    # train_labels, val_labels = y_aug[train_index], y_aug[val_index]
    cross_ind.append(val_index)

cross_ind = np.array(cross_ind)
print(cross_ind)

/home/lennart/.local/mambaforge/envs/ml/lib/python3.10/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=6.
  warnings.warn(


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (6,) + inhomogeneous part.

In [ ]:
print(len(X_aug[cross_ind[0]]))
labels_1_6 = y_aug[cross_ind[0]]
print(labels_1_6.sum(axis = 0))

2001
[333 334 333 333 333 334]


In [ ]:
np.save('../data/cross_val_6fold_2000.npy', cross_ind)

In [14]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold

# Load data
X = np.load('../data/data_aug_2000.npy')
y = pd.read_csv('../data/labels_aug_2000.csv', delimiter=',', index_col=0).to_numpy()

# Generate labels
labels_aug = []
for i in range(len(y)):
    if 1 in y[i]:
        k = np.argmax(y[i])
        labels_aug.append(k)
    else:
        labels_aug.append(7)

# Create KFold object
kf = KFold(n_splits=10)

# Ensure the directory exists
output_dir = "../data/6_fold/"
os.makedirs(output_dir, exist_ok=True)

# Split into 10 folds and save
for i, (train_index, test_index) in enumerate(kf.split(X, labels_aug)):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    val_size = int(0.2 * X_train.shape[0])
    X_val = X_train[:val_size]
    y_val = y_train[:val_size]
    X_train = X_train[val_size:]
    y_train = y_train[val_size:]

    np.save(os.path.join(output_dir, f"X_train_fold{i}.npy"), X_train)
    np.save(os.path.join(output_dir, f"y_train_fold{i}.npy"), y_train)
    np.save(os.path.join(output_dir, f"X_val_fold{i}.npy"), X_val)
    np.save(os.path.join(output_dir, f"y_val_fold{i}.npy"), y_val)
    np.save(os.path.join(output_dir, f"X_test_fold{i}.npy"), X_test)
    np.save(os.path.join(output_dir, f"y_test_fold{i}.npy"), y_test)

print("Data split into 10 folds")

OSError: 103728000 requested and 100732889 written